In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

Unser Datensatz ist eine 4x4 Matrix, d.h. dass wir vier Dimensionen/Merkmale und vier Datenpunkte haben.

In [11]:
X = np.array([
    [[1., 0., 0., 0.]],
    [[0., 1., 0., 0.]],
    [[0., 0., 1., 0.]],
    [[0., 0., 0., 1.]],
            ]        
    )

In [3]:
# Create models
inputs = Input(shape=(X.shape[1],X.shape[2]))
encoded = Dense(2, activation='sigmoid')(inputs)
decoded = Dense(4, activation='sigmoid')(encoded)

# separate model for autoencoder and encoder only
encoder = Model(inputs, encoded)
autoencoder = Model(inputs, decoded)

In [4]:
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 4)]            0         
_________________________________________________________________
dense (Dense)                (None, 1, 2)              10        
_________________________________________________________________
dense_1 (Dense)              (None, 1, 4)              12        
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________


In [13]:
autoencoder.fit(X, X, epochs=3_000, verbose=0)

In [6]:
# Predict the train data with autoencoder
x_hat = np.around(autoencoder.predict(X))
x_hat

array([[[1., 0., 0., 0.]],

       [[0., 1., 0., 0.]],

       [[0., 0., 1., 0.]],

       [[0., 0., 0., 0.]]], dtype=float32)

Wie oben zu sehen ist, entspricht unser Ausgabesignal $\mathbf{\hat{x}}$ exakt unserem Eingabesignal $\mathbf{x}$. Auch bei näherer Betrachtung des Encoders ist genau das eingetroffen, was wir uns vorher überlegt haben: unser Modell, hat unser Eingangssignal mit vier Dimensionen in ein Signal mit nur zwei Dimensionen umgewandelt!

In [7]:
# Predict the train data with encoder only
np.around(encoder.predict(X))

array([[[0., 1.]],

       [[1., 0.]],

       [[1., 1.]],

       [[0., 1.]]], dtype=float32)

In [8]:
# Encoder part with classifier
classify = Dense(4, activation='softmax')(encoded)

classifier = Model(inputs, classify)
classifier.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
classifier.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 4)]            0         
_________________________________________________________________
dense (Dense)                (None, 1, 2)              10        
_________________________________________________________________
dense_2 (Dense)              (None, 1, 4)              12        
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________


In [14]:
classifier.fit(X, np.array([0, 1, 2, 3]), epochs=3_000, verbose=0)

In [10]:
# Predicted label of the encoder and classifier model
[np.argmax(classifier.predict(np.array([X[i]]))) for i in range(0,4)]

[0, 1, 2, 3]